# Warm up
Let's start with some warm up exercises to get you familiar with the database and do some SQL querying.
Make sure to have downloaded the NBA dataset form Kaggle as described in the README.

In [ ]:
import sqlite3 as sql
import pandas as pd
from pathlib import Path

## TODO: Use pathlib to get the path to the data directory. Path(__file__) won't work in a Jupyter notebook.
DATA_PATH = Path.cwd() / 'data'/ 'nba.sqlite'

## Create a SQLite connection and run some SQL queries

In [ ]:
con = sql.connect(DATA_PATH)  # connect to the database
query = 'SELECT * FROM game LIMIT 10'  # write a query that selects the first 10 rows from the game table
top_10_game = pd.read_sql(query, con)  # read the query into a pandas dataframe
top_10_game.head()  # print the dataframe

In [ ]:
# let's look at the schema of the database
# TODO: sqlite_master is the name of the table you want to inspeact: Replace TABLE_NAME with the name of the table you want to inspect
query = """
    SELECT *
    FROM sqlite_master
"""

pd.read_sql(query, con).sort_values('name')

In [ ]:
# let's see all the tables we have in the database
# TODO: Filter for tables only
query = """
    SELECT name
    FROM sqlite_master
    WHERE type = 'table'
"""
pd.read_sql(query, con)

In [ ]:
# let's have a look at the columns in the game table
query = """
    PRAGMA table_info(draft_combine_stats)
"""
pd.read_sql(query, con)

In [ ]:
# let's find the earliest recorded game
query = """
    SELECT game_date
    FROM game
    ORDER BY game_date
    LIMIT 1
"""

print(pd.read_sql(query, con))

# let's find the latest recorded game
query = """
    SELECT game_date
    FROM game
    ORDER BY game_date DESC
    LIMIT 1
"""

print(pd.read_sql(query, con))


In [ ]:
# TODO: Find four interesting facts about the data and write a query to find the answer

In [ ]:
# Fact 1: Which teams have the most home wins in NBA history?
# 🏀 事实1：哪支球队主场胜率最高？（前10名）

print("=" * 60)
print("FACT 1: Top 10 Teams with Most Home Wins")
print("=" * 60)

query = """
    SELECT 
        team_name_home as team_name,
        COUNT(*) as total_home_wins
    FROM game
    WHERE wl_home = 'W'
    GROUP BY team_name_home
    ORDER BY total_home_wins DESC
    LIMIT 10
"""

result = pd.read_sql(query, con)
print(result)
print("\nInsight: The teams listed above dominated at their home courts!")

In [ ]:
# Fact 2: What was the highest-scoring game in NBA history?
# 📊 事实2：NBA历史上得分最高的比赛

print("\n" + "=" * 60)
print("FACT 2: Highest-Scoring Game in NBA History")
print("=" * 60)

query = """
    SELECT 
        team_name_home,
        team_name_away,
        pts_home,
        pts_away,
        (pts_home + pts_away) as total_points,
        game_date
    FROM game
    ORDER BY total_points DESC
    LIMIT 1
"""

result = pd.read_sql(query, con)
print(result)
print(f"\nInsight: The game had a combined {result['total_points'].values[0]} points!")

In [ ]:
# 这告诉我们什么？

# 🌟 这是全明星赛！（不是常规赛）
#  全明星赛是表演性质，防守松懈，所以得分特别高
#  常规赛很少有超过250分的比赛

# 有趣的发现：这个结果揭示了一个重要信息：如果你想找"真正的高分比赛"，应该排除全明星赛！

In [ ]:
# Excluding the highest-scoring game of the All-Star Game
# 改进版 Fact 2: 排除全明星赛的最高分比赛（这样会更有意义）

print("\n" + "=" * 60)
print("FACT 2 (Improved): Highest-Scoring Regular/Playoff Game")
print("(Excluding All-Star Games)")
print("=" * 60)

query = """
    SELECT 
        team_name_home,
        team_name_away,
        pts_home,
        pts_away,
        (pts_home + pts_away) as total_points,
        game_date
    FROM game
    WHERE team_name_home NOT LIKE '%All Star%'
      AND team_name_away NOT LIKE '%All Star%'
    ORDER BY total_points DESC
    LIMIT 1
"""

result = pd.read_sql(query, con)
print(result)
print(f"\nInsight: This was a real game (not All-Star), with {result['total_points'].values[0]} total points!")
print("Much more impressive than an exhibition game!")
print("\n🏀 Historical Context:")
print("This game went to TRIPLE OVERTIME (3 extra periods)!")
print("It represents the fast-paced, high-scoring era of 1980s basketball.")
print("Modern NBA rarely sees games with 260+ combined points.")

In [ ]:
# Comments & Insights:
# 这场比赛发生在 1983年12月13日，是NBA历史上最疯狂的比赛之一！

# (i). 为什么这么特殊？
    # 1.这场比赛打了3个加时赛！（triple overtime）
    # 2.最终 Detroit Pistons 以 186:184 险胜
    # 3. 这是非全明星赛中得分最高的比赛

# (ii). 篮球战术的变化 📈
    # 1. 1980年代的NBA节奏非常快，防守相对松散
    # 2. 对比现在：现代NBA很少有单场超过260分的比赛
    # 3. 这反映了篮球从"快速进攻"到"战术防守"的演变

# (iii). 惊人的统计 📊
    # 1. 平均每队得分 185分
    # 2. 如果是正常48分钟比赛，这相当于每节得分46分！
    # 3. 但因为有3个加时（总共63分钟），平均每分钟得分约 5.9 分

# (iv). 对比全明星赛 🌟
    # 还记得之前的全明星赛吗？（374分，2017年）
    # 全明星赛：374分 → 表演性质，几乎不防守
    # 这场比赛：370分 → 真刀真枪的竞技！
    # 结论：排除全明星后，这个结果更有说服力！


In [ ]:
# Fact 3: Which players had the longest NBA careers?
# 👴 事实3: 哪些球员的NBA生涯最长?

print("\n" + "=" * 60)
print("FACT 3: Players with Longest NBA Careers")
print("=" * 60)

query = """
    SELECT 
        full_name,
        from_year,
        to_year,
        (to_year - from_year) as career_length_years
    FROM player
    WHERE to_year IS NOT NULL 
      AND from_year IS NOT NULL
    ORDER BY career_length_years DESC
    LIMIT 10
"""

result = pd.read_sql(query, con)
print(result)
print("\nInsight: These players showed incredible longevity and dedication!")
print("Playing 15+ years in the NBA is extremely rare!")

In [ ]:
# 上述问题报错：player 表里没有 from_year 这一列！而上述代码假设了 player 表有这些列。
# 所以实际上这个数据库的结构可能不同！
# ✅ 解决方案：先查看表结构 —— 看看 player 表有哪些列 (下述代码跑出后会显示 player 表的所有列名！)：

# Let's check what columns are in the player table
print("Checking columns in 'player' table:")
query = """
    PRAGMA table_info(player)
"""
result = pd.read_sql(query, con)
print(result)

In [ ]:
# 上述 player 表显示：确实没有 from_year 和 to_year！
# 备用方案：用其他表来找职业生涯信息 —— 如果 player 表没有年份信息，我们可以用 common_player_info 表：

# Fact 3: Players with Longest NBA Careers (Alternative approach)
# 👴 事实3: NBA生涯最长的球员

print("\n" + "=" * 60)
print("FACT 3: Players with Most Seasons Played")
print("=" * 60)

query = """
    SELECT 
        display_first_last as player_name,
        from_year,
        to_year,
        (to_year - from_year) as seasons_played
    FROM common_player_info
    WHERE from_year IS NOT NULL 
      AND to_year IS NOT NULL
      AND from_year > 0
    ORDER BY seasons_played DESC
    LIMIT 10
"""

result = pd.read_sql(query, con)
print(result)
print("\nInsight: These legends played 15+ seasons - incredible longevity!")
print("Playing into your late 30s or even 40s requires exceptional dedication and fitness.")

In [ ]:
# 上述表结果出现明显异常：对于0号球员，55年的职业生涯？不可能！
    # ——> NBA球员平均职业生涯只有4-5年，最长的也就是20多年（比如 Vince Carter）。

# 可能的原因：
    #  1. `from_year` 和 `to_year` 可能不是指"职业生涯"
    #  2. 可能是"出生年份"到"某个统计年份"
    #  3. 或者数据本身有错误

# 但是好消息：有些数据看起来是对的！
    #  Vince Carter:    1998-2019, 21 seasons  ✅ (他确实打了22年！)    
    #  Kevin Garnett:   1995-2015, 20 seasons  ✅ (正确！)
    #  Dirk Nowitzki:   1998-2018, 20 seasons  ✅ (正确！)
    #  Kareem Abdul-Jabbar: 1969-1988, 19 seasons ✅ (他打了20年！)
    #  ——> 这些是真正的传奇！


# ✅ 改进方案：过滤掉异常数据
# Fact 3 (Improved): Players with Longest NBA Careers
# 事实3（改进版）: NBA生涯最长的球员

print("\n" + "=" * 60)
print("FACT 3: Players with Longest NBA Careers")
print("=" * 60)

query = """
    SELECT 
        display_first_last as player_name,
        from_year,
        to_year,
        (to_year - from_year) as seasons_played
    FROM common_player_info
    WHERE from_year IS NOT NULL 
      AND to_year IS NOT NULL
      AND from_year > 1946                          -- NBA founded in 1946
      AND (to_year - from_year) <= 25               -- Filter out 55-year anomalies
      AND (to_year - from_year) >= 10               -- Only long careers
    ORDER BY seasons_played DESC
    LIMIT 10
"""

result = pd.read_sql(query, con)
print(result)
print("\nInsight: These legends played 15-22 seasons - incredible longevity!")
print("Playing into your late 30s requires exceptional fitness and skill.")
print(f"The longest career was {result['seasons_played'].values[0]:.0f} seasons by {result['player_name'].values[0]}!")
print("Note: Some players' data may include international leagues.")

In [ ]:
# 🏀 上图示结果显示 Top 10 职业生涯最长的球员

# 🏆 #1: Andrew Gaze - 24 赛季 (1993-2017, 24 yrs)
    # 🇦🇺 澳大利亚传奇球员
    # 但要注意：他在NBA实际只打了7年左右
    # 这24年可能包含了他在澳大利亚NBL联赛和其他国际联赛的时间
    # 数据异常？ 可能是数据库把他所有职业生涯都算进去了
    # 🤔 问题： 这个数据可能不完全准确，因为他不是纯NBA生涯24年; 
    # 但是除了 Andrew Gaze 可能有小问题外，其他都准确！

# 🌟 #2: Kevin Willis - 22 赛季 (1984-2006, 22 yrs)
    # 💪 真正的"铁人"！纯正的NBA生涯22年
    # 44岁还在打球（NBA历史上最老的球员之一）
    # 生涯总共打了1424场比赛
    # 特点： 身体素质超强，职业态度极佳

# 🦅 #3: Vince Carter - 21 赛季 (1998-2019, 21 yrs)
    # 🔥 "半人半神"（绰号：因为他的扣篮太夸张）
    # 实际打了22个赛季（1998-2020）
    # 从暴力扣篮王到三分射手的华丽转型
    # 43岁退役，跨越了4个十年代！
    # 传奇时刻： 2000年扣篮大赛，史上最精彩

# 💎 #4: Kevin Garnett - 20 赛季 (1995-2015, 20 yrs)
    # 🏆 2008年帮助凯尔特人夺冠
    # 防守大师，15次入选全明星
    # 特点： 激情、领导力、全能
    # 从高中直接进入NBA的传奇之一

# 🔥 #5: Udonis Haslem - 20 赛季 (2003-2023, 20 yrs)
    # 🏆 帮助热火赢得3次总冠军
    # 忠诚典范： 整个职业生涯都在迈阿密热火！
    # 后期主要是替补和导师角色
    # 2023年43岁退役

# 🇩🇪 #6: Dirk Nowitzki - 20 赛季 (1998-2018, 20 yrs)
    # 🏆 2011年带领独行侠夺冠（击败热火三巨头）
    # 革命性球员： 7尺长人能投三分，改变了篮球
    # 整个生涯都在达拉斯，忠诚传奇
    # 14次全明星，MVP获得者

# 👑 #7: Robert Parish - 20 赛季 (1976-1996, 20 yrs)
    # 🏆 80年代凯尔特人王朝的核心（和Larry Bird搭档）
    # 绰号"酋长"（The Chief）
    # 43岁还在打球
    # NBA历史出场次数第二多（1611场）

# 🐐 #8: Kareem Abdul-Jabbar - 19 赛季 (1969-1988, 19 yrs)
    # 🏆 6次总冠军，6次MVP
    # NBA历史得分王（38,387分，直到2023年才被LeBron打破）
    # 标志性技术： 勾手投篮（Sky Hook），无人能防
    # 退役时42岁

# 🐍 #9: Kobe Bryant - 19 赛季 (1996-2015, 19 yrs)
    # 🏆 5次总冠军（全部在湖人）
    # 曼巴精神： 极致的训练态度和求胜欲
    # 81分单场得分（NBA历史第二高）
    # 2020年不幸因直升机事故去世，全球哀悼

In [ ]:
# Fact 4 (Improved): Which season had the highest average points per game?
# 🗓️ 事实4（改进版）: 哪个赛季的平均得分最高?

print("\n" + "=" * 60)
print("FACT 4 (Improved): Season with Highest Average Points")
print("=" * 60)

query = """
    SELECT 
        season_id,
        AVG(pts_home + pts_away) as avg_total_points,
        COUNT(*) as total_games
    FROM game
    WHERE team_name_home NOT LIKE '%All Star%'
      AND team_name_away NOT LIKE '%All Star%'
    GROUP BY season_id
    HAVING COUNT(*) > 100
    ORDER BY avg_total_points DESC
    LIMIT 10
"""

result = pd.read_sql(query, con)
print(result)
print("\n🏀 Basketball History Insights:")
print("The 1960s-70s were the highest-scoring era in NBA history!")
print("Average games had 230+ combined points - that's 115 per team!")
print("Modern basketball (2020s) is faster again, but still not as high-scoring.")

In [ ]:
#💡 关键 Insights（Fact 4 这个结果告诉我们什么）

# Insight 1: 1960-70年代是得分最高的时代！
    # 得分最高的赛季平均每场 233.4分    
    # 这是NBA的"黄金时代"，节奏非常快

# Insight 2: 篮球风格的演变
    # 1960s-70s: 233分/场 → 快速进攻，高得分
    # 1990s-2000s: 190-200分/场 → 防守加强
    # 2010s-2020s: 220分/场 → 又开始提速（三分球时代）

# Insight 3: 比赛数量的变化
    # 有些赛季只有360场比赛（可能是停摆赛季）
    # 正常赛季有1000+场比赛